In [3]:
import os
import sys
import locale
import re
from dotenv import load_dotenv, find_dotenv
import psycopg2

load_dotenv(find_dotenv(filename='MF.env'))
locale.setlocale(locale.LC_COLLATE, "lt_LT.UTF-8")
z_dir = os.getenv("z_dir")
opus_dir = os.getenv("opus_dir")

conn = psycopg2.connect(
    dbname=os.getenv("database"),
    user=os.getenv("user"),
    password=os.getenv("password"),
    host=os.getenv("host"),
    port=os.getenv("port")
) 

cur = conn.cursor()

In [109]:
def read_chunks(file_name):
    file_name = f"{file_name}.md"
    chunks_path = os.path.join(z_dir, file_name)
    chunks = []
    with open(chunks_path, "r", encoding="utf-8") as file:
        chunks += [line.strip().split(", ") for line in file if line.strip()]
        return chunks

def append_words(file_name, words):
    file_path = os.path.join(z_dir, f"{file_name}.md")
    with open(file_path
    , "a", encoding="utf-8") as file:
        stringed_words = "\n\n".join(words)
        if stringed_words: stringed_words = "\n\n" + stringed_words
        file.write(stringed_words)
    return

def empty_file(file_name):
    file_name_ = f"{file_name}.md"
    backup_name = f"{file_name}_backup.md"
    chunks_path = os.path.join(z_dir, file_name_)
    backup_path = os.path.join(z_dir, backup_name)
    with open(chunks_path, "r", encoding="utf-8") as file:
        with open(backup_path, "w", encoding="utf-8") as backup:
            backup.write(file.read())
    with open(chunks_path, "w", encoding="utf-8") as file:
        file.write("[[()_Rėpas]]")
        return None

In [82]:
cores = read_chunks("cores")
cores = [item for item in cores if "delete" not in item]

In [83]:
to_core_pairs = tuple((item[0], core) for item in cores for core in item[1:])

In [85]:
new_morphs = tuple((item[1],) for item in to_core_pairs)

In [86]:
insert_morph_to_morphs_query = f"""
INSERT INTO morphs (morph, entered)
VALUES (%s, 1)
ON CONFLICT (morph) DO UPDATE SET entered = morphs.entered + 1
RETURNING id;
"""

In [87]:
cur.executemany(insert_morph_to_morphs_query, new_morphs)

In [88]:
conn.commit()

In [89]:
to_core_pairs

(('abejingo', 'abejoti'),
 ('abstraktus', 'abstraktus'),
 ('abudu', 'du'),
 ('acetilakrilatas', 'acetilakrilatas'),
 ('adamkus', 'valdasadamkus'),
 ('adata', 'adata'),
 ('adrenalinas', 'adrenalinas'),
 ('afganistano', 'afganistanas'),
 ('afrodiziakas', 'afrodiziakas'),
 ('agnostikas', 'agnostika'),
 ('agregacija', 'agreguoti'),
 ('aguona', 'aguona'),
 ('aguonėlė', 'aguona'),
 ('aguonos', 'aguona'),
 ('aguonpienis', 'aguona'),
 ('aguonpienis', 'pienas'),
 ('agurkas', 'agurkas'),
 ('agurkėlis', 'agurkas'),
 ('aidu', 'aidas'),
 ('aikčiot', 'aikčioti'),
 ('aikčioti', 'aikčioti'),
 ('aikjų', 'aikjų'),
 ('aikosas', 'aikosas'),
 ('aikštėn', 'aikštė'),
 ('aiškiaregys', 'aišku'),
 ('aiškiaregys', 'regėti'),
 ('aišku', 'aišku'),
 ('aistringai', 'aistra'),
 ('aistringo', 'aistra'),
 ('aistruolis', 'aistra'),
 ('aitvarai', 'aitvaras'),
 ('aitvaras', 'aitvaras'),
 ('akacija', 'akacija'),
 ('akis', 'akis'),
 ('akių', 'akis'),
 ('akivaizdu', 'akivaizdu'),
 ('aklas', 'aklas'),
 ('aklus', 'aklas'),
 ('

In [90]:
morphs = tuple(item[0] for item in to_core_pairs)

In [91]:
cur.execute(f'SELECT morph, id FROM morphs WHERE morph in %s', (morphs,))
morph_morph_id = cur.fetchall()

In [92]:
morph_to_morph_id = {item[0]: item[1] for item in morph_morph_id}

In [93]:
cores = tuple(item[1] for item in to_core_pairs)

In [100]:
cur.execute(f'SELECT core, id FROM cores WHERE core in %s', (cores,))
core_core_id = cur.fetchall()

In [127]:
core_to_core_id = {item[0]: item[1] for item in core_core_id}

In [102]:
the_thing = tuple((morph_to_morph_id[item[0]], core_to_core_id[item[1]]) for item in to_core_pairs)

In [106]:
cur.executemany(f'INSERT INTO morphs_cores (morph_id, core_id) VALUES (%s, %s) ON CONFLICT DO NOTHING', the_thing)
conn.commit()

In [132]:
core_to_core_id

{'daržovė': 397,
 'rimas': 2176,
 'gesti': 2658,
 'krautuvė': 1076,
 'ankštis': 5,
 'baklažanas': 6,
 'bulvė': 7,
 'asilas': 8,
 'amplitudė': 9,
 'akvalangas': 10,
 'akivaizdu': 11,
 'alkaridas': 12,
 'antiseptikas': 13,
 'aorta': 14,
 'architektas': 15,
 'architektūra': 16,
 'budulis': 17,
 'bunkeris': 18,
 'defakto': 19,
 'dozė': 20,
 'guoga': 21,
 'gyvulys': 22,
 'acetilakrilatas': 23,
 'antropologas': 24,
 'bobslėjus': 25,
 'alimentai': 26,
 'artifaktas': 27,
 'arba': 28,
 'baidyti': 29,
 'balandis': 30,
 'birželis': 31,
 'alfa': 32,
 'baltymas': 33,
 'bedantis': 34,
 'epilogas': 35,
 'benzinas': 36,
 'boratas': 37,
 'boš': 38,
 'haki': 39,
 'karsti': 40,
 'amžinas': 41,
 'apetitas': 42,
 'arbūzas': 43,
 'arka': 44,
 'banglas': 45,
 'barbaras': 46,
 'beretė': 47,
 'buožė': 48,
 'asas': 49,
 'apgailėtinas': 50,
 'amoniakas': 51,
 'apskritimas': 52,
 'akordeonas': 53,
 'atletas': 54,
 'bagažas': 55,
 'bastytis': 56,
 'abstraktus': 57,
 'aikosas': 58,
 'agurkas': 59,
 'anakonda': 60,


In [ ]:
core_thing = tuple((core_to_core_id[core], core_to_core_id[core]) for core in core_to_core_id)

In [137]:
cur.execute(f"SELECT morph_id, id FROM cores")
core_core = cur.fetchall()

In [140]:
cur.executemany(f'INSERT INTO morphs_cores (morph_id, core_id) VALUES (%s, %s) ON CONFLICT DO NOTHING', core_core)
conn.commit()

In [141]:
cur.execute("SELECT morph FROM MORPHS WHERE id NOT in (SELECT morph_id FROM morphs_cores)")
cur.fetchall()

[('mažolavašosupikantiškupaprašo',),
 ('bizynes',),
 ('dadu',),
 ('q\t',),
 ('bingobangobongobišbašboš',),
 ('nekilmingo',),
 ('cėpėtėesdė',),
 ('čikuraku',),
 ('plaukt',),
 ('auplyz',),
 ('čingadomadreputas',),
 ('pavyzdžiai',),
 ('tekriukteli',),
 ('čekinas',)]

In [153]:
cur.fetchall()

[('bulviųsalotos',), ('daržovė',)]

In [152]:
cur.execute(f"SELECT morph FROM morphs WHERE morph in %s", (("daržovė", "bulviųsalotos"),))

In [111]:
patterns = read_chunks("blokai")

In [114]:
patterns = tuple((s[0].partition("~")[0]).lower() for s in patterns[1:])

In [116]:
patterns = patterns[:-1]

In [123]:
patterns = tuple((item,) for item in patterns)

In [124]:
patterns

(('yb',),
 ('[šž][tvmknlpg]?[iė][sš]$',),
 ('š(e|ę|ia)s$',),
 ('or',),
 ('oga$',),
 ('ogas$',),
 ('ogis$',),
 ('og',),
 ('p[aęe]s',),
 ('š[ae]is',),
 ('utės',),
 ('uos$',),
 ('uos',),
 ('als(u|o|io)',),
 ('us$',),
 ('(č|dž)iau',),
 ('als',),
 ('(k|m)?als',),
 ('uot',),
 ('ož',),
 ('či(ų|u)',),
 ('iek',),
 ('t[o|e]li$',),
 ('il(n|m)(ų|o)$',),
 ('dži[uų]$',),
 ('ik[r|l][aąe]$',),
 ('ėd',),
 ('ėd[^(av)]',),
 ('atu',),
 ('eik(ia|e)',),
 ('ynas',),
 ('yn',),
 ('vų',),
 ('ysiu$',),
 ('rog',),
 ('van',),
 ('l[eėi]tas$',),
 ('d[eė]tas$',),
 ('m[eė]tas$',),
 ('g[eė]l',),
 ('atl',),
 ('tle',),
 ('l(ie|ė)tas$',),
 ('l(ie)tas$',),
 ('l(e|ė)tas$',),
 ('gyv',),
 ('vat[ėe]$',),
 ('rat[ėe]$',),
 ('at[ėe]$',),
 ('yv',),
 ('g[yi][vrkl]',),
 ('ara$',),
 ('era',),
 ('era$',),
 ('ven',),
 ('pir',),
 ('^pir',),
 ('kir',),
 ('bur',),
 ('kur',),
 ('uog',),
 ('uo[mn]is',),
 ('uolis$',),
 ('akt(as|is)',),
 ('akt(as|is|a)',),
 ('eit',),
 ('rai$',),
 ('oja$',),
 ('ažas',),
 ('aža',),
 ('(?<!i)(ia|e)žas',),
 ('ežt

In [125]:
cur.executemany(f"INSERT INTO patterns (pattern) VALUES (%s)", patterns)


In [126]:
conn.commit()

In [121]:
conn.rollback()

In [ ]:
select_morph_id_core_id_query = f"""
SELECT m.id, c.morph_id
FROM morphs m
LEFT JOIN morphs_clusters mc
ON m.id = mc.morph_id
LEFT JOIN cores c
ON mc.cluster_id = c.id
WHERE m.morph in %s
"""

In [160]:
cur.execute(select_morph_id_core_id_query, (("daržovė",),))

UndefinedColumn: column c.morph_id does not exist
LINE 2: SELECT m.id, c.morph_id
                     ^
HINT:  Perhaps you meant to reference the column "mc.morph_id".


In [164]:
cur.execute(f"select morph_id from cores")
cur.fetchall()

[(24036,),
 (7544,),
 (4511,),
 (9425,),
 (12609,),
 (8500,),
 (977,),
 (7538,),
 (1001,),
 (1140,),
 (1303,),
 (1486,),
 (7707,),
 (1547,),
 (1561,),
 (8530,),
 (8529,),
 (1648,),
 (8997,),
 (8556,),
 (1925,),
 (1635,),
 (1729,),
 (1894,),
 (1904,),
 (2053,),
 (7914,),
 (2212,),
 (2609,),
 (7931,),
 (2620,),
 (2499,),
 (2636,),
 (5546,),
 (8005,),
 (2289,),
 (1545,),
 (8046,),
 (5591,),
 (5617,),
 (5781,),
 (4166,),
 (2158,),
 (4890,),
 (8187,),
 (8111,),
 (9047,),
 (8624,),
 (8105,),
 (6018,),
 (4502,),
 (783,),
 (6873,),
 (2248,),
 (6945,),
 (1519,),
 (1282,),
 (1567,),
 (8448,),
 (9255,),
 (9954,),
 (9786,),
 (10159,),
 (9028,),
 (10563,),
 (10647,),
 (1825,),
 (10603,),
 (10577,),
 (3089,),
 (10700,),
 (10596,),
 (10565,),
 (10619,),
 (10694,),
 (10545,),
 (10583,),
 (10690,),
 (10582,),
 (10622,),
 (10651,),
 (8068,),
 (10689,),
 (10634,),
 (10566,),
 (9065,),
 (4169,),
 (8143,),
 (12789,),
 (12790,),
 (12791,),
 (12801,),
 (8463,),
 (12806,),
 (12811,),
 (12817,),
 (1364,),
 (12

In [162]:
conn.rollback()